In [1]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")
include(pathtorepo * "dynamical-systems\\box_square_functions.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie, LinearAlgebra, LinearSolve

pathtofile = "C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab\\"
I0_hom = load(pathtofile * "I0_hom_hom.jld")["data"]
u0_hom = load(pathtofile * "U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
index_point_from_curve = 1

const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305; const U0 = u0_hom[index_point_from_curve]

t = 2000.0; tspan = [0.0, t]
u0 = [0.0, 0.0, 0.0]
ϵ_box = 1.0e-5; ϵ_shift = 1.0e-2;

In [3]:
number_points_on_side = 10
total_count_points = number_points_on_side * 2

I0start = I0_hom[index_point_from_curve]
I0end = I0_hom[index_point_from_curve] + ϵ_shift
len_range = 5

p_range =  range(I0start, I0end, length = len_range)

size_vector = len_range * total_count_points;

In [4]:
dots_u0 = zeros(size_vector, 3)
check_events = Vector{Bool}(undef, size_vector)
time_events = zeros(size_vector)
dots_on_event = zeros(size_vector, 3)
αs = zeros(size_vector, 3)
norms = zeros(size_vector, 3)

100×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [5]:
for p_index in range(1, length(p_range), step = 1)

    if p_index == 1
        global start_internal_index = 1
    end

    println("index param: $(p_index)"); flush(stdout)

    p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, p_range[p_index]]

    fixedpoint = get_fixed_point(TM, jacob_TM_, p, u0)
    A = get_matrix(fixedpoint[1], p, jacob_TM_, 2)

    dots_right_side, array_α_vec_right,
    dots_left_side, array_α_vec_left,
    dots_up_side, array_α_vec_up,
    dots_down_side, array_α_vec_down = get_arrays_dots(number_points_on_side, 3)

    fill_side_square(ϵ_box, fixedpoint[1], number_points_on_side, A, 
    dots_right_side, array_α_vec_right, dots_left_side, array_α_vec_left,
    dots_up_side, array_α_vec_up, dots_down_side, array_α_vec_down)


    condition = make_event(fixedpoint[1], ϵ_box, A)
    cb = ContinuousCallback(condition, nothing, affect!)

    up_and_down_side = cat(dots_up_side, dots_down_side, dims = 1)
    final_internal_index = total_count_points * p_index

    println("start internal index: $(start_internal_index)"); flush(stdout)
    println("final internal index: $(final_internal_index)"); flush(stdout)

    start_internal_index = trajectory_from_sides(p, A, up_and_down_side, final_internal_index,
                    fixedpoint[1], cb, start_internal_index)
                    
    println("-------------------------------"); flush(stdout)

end

index param: 1


start internal index: 1


final internal index: 20


-------------------------------


index param: 2


...

-------------------------------


index param: 5


start internal index: 81


final internal index: 100


-------------------------------


In [23]:
dots_u0[6, :]

3-element Vector{Float64}:
 8.345846373031147
 0.7384874228485434
 0.43829757648368717

In [26]:
fixedpoint

UndefVarError: UndefVarError: `fixedpoint` not defined

In [24]:
get_norms(dots_u0[6, :], fp[1], A)

UndefVarError: UndefVarError: `fp` not defined

In [6]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

number of event: 4; number of no event: 96


In [17]:
αs[6, :]

3-element Vector{Float64}:
  1.0123496859795507e-5
  2.7813591103930512e-5
 -6.45014459268911e-6

In [11]:
αs_without_zeros =  permutedims(stack(filter(!iszero,eachrow(αs))))
# last points in space (α2, α3)
f = Figure(resolution = (700, 700))
ax = Axis(f[1, 1], xlabel = L"α_2", ylabel = L"α_3", xlabelsize = 35, ylabelsize = 35)
scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, αs_without_zeros[2, 2], αs_without_zeros[2, 3], markersize = 5)

display(GLMakie.Screen(), f);

In [18]:
norms[6]

2.7813591103930512e-5

In [12]:
αs_without_zeros[2, :]

3-element Vector{Float64}:
  1.0123496859795507e-5
  2.7813591103930512e-5
 -6.45014459268911e-6

In [8]:
filename = "index_point_from_curve $(index_point_from_curve)"
filename = filename*".jld2"

"index_point_from_curve 1.jld2"

In [9]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\manifolds\\TM\\box square\\")
jldsave(filename; dots_u0, check_events, time_events, dots_on_event ,αs, norms)